# Initiate a in place restore of a MSSQL Database
In Rubrik, a restore is a in place operation. The original database on the original SQL Server instance is overwritten with the point in time provided. This is very important to understand as you could cause a production outage recovering a database when you mean to make a copy of a database. For that use export. Restore should only be used as a disaster recovery process.  

New-RscMssqlRestore has multiple parameters, but the important ones to understand for most workflows are:
- RscMssqlDatabase: This will be the object returned from Get-RscMssqlDatabase
- Recovery Time: For the point in time value, there are a few different possibilities. 
    - **latest**: This will tell Rubrik to export the database to the latest recovery point Rubrik knows about. This will include the last full and any logs to get to the latest recovery point
    - **lastfull**: This will tell Rubrik to restore back to the last full backup it has
    - **RestoreTime**: This can be done in 1 of 2 formats
        - Format: (HH:MM:SS.mmm) or (HH:MM:SS.mmmZ) - Restores the database back to the local or UTC time (respectively) at the point in time specified within the last 24 hours
        - Format: Any valid <value> that PS Get-Date supports in: "Get-Date -Date <Value>"
            - Example: "2018-08-01T02:00:00.000Z" restores back to 2AM on August 1, 2018 UTC.
    This cmdlet will run Get-RscMssqlDatabaseRecoveryPoint
- FinishRecovery: Ommitting this switch will restore the database into NORECOVERY mode. 

# Workflow
To get information about the SQL Server Instance, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **New-RscMssqlRestore** to start an in place restore of the database to the point in time provided. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC
 new-rscmssqllogBackup -RscMssqlDatabase $mssqldatabase  

## Connect to RSC

In [2]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC


Loading Rubrik Security Cloud PowerShell Module (PowerShell Core)...

Welcome to the Rubrik Security Cloud PowerShell SDK!

	Please refer to the Getting Started section of the Readme documentation: 
	https://github.com/rubrikinc/rubrik-powershell-sdk#getting-started


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20230920-45     v20231011-41



## Get the Cluster ID

In [3]:
$ClusterName = "vault-r-london"
$RubrikCluster = Get-RscCluster -Name $ClusterName

Get-RscCluster field profile: DEFAULT


## Get the Instance Information

In [4]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscMssqlInstance = Get-RscMssqlInstance -HostName $HostName  -ClusterId $RubrikCluster.id

Get-RscMssqlInstance field profile: DEFAULT


## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2017"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

Get-RscMssqlDatabase field profile: DEFAULT


## Intiate an in place restore of a database

In [6]:
New-RscMssqlRestore -RscMssqlDatabase $RscMssqlDatabase -Latest -FinishRecovery

Get-RscMssqlDatabaseRecoveryPoint field profile: DEFAULT
MethodInvocationException: /Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk/output/Toolkit/Public/New-RscMssqlRestore.ps1:103
Line |
 103 |          $query.Invoke()
     |          ~~~~~~~~~~~~~~~
     | Exception calling "Invoke" with "0" argument(s): " One or more errors occurred. (The request
     | generated an error.  Query: mutation MutationRestoreMssqlDatabase($input:
     | RestoreMssqlDatabaseInput!) {   restoreMssqlDatabase   (   input: $input   )   {   id  
     | nodeId   status    __typename }  __typename }  Variables:
     | {"input":{"config":{"finishRecovery":true,"maxDataStreams":2,"recoveryPoint":{"date":"2023-10-18T18:14:32-04:00"}},"id":"62e52b73-0516-5698-8c62-968f3c664dda"}} Operation Name: MutationRestoreMssqlDatabase   GraphQLResponse<JObject>: Data: null Errors:  500 UNKNOWN: Status: 422 Unprocessable Entity. Body: {"erro

## Track the status of the restore

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc